<a href="https://colab.research.google.com/github/mounsifelatouch/cdd/blob/master/notebooks/3_cdd_ml_part_3_descriptor_dataset_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bioinformatics Project - Computational Drug Discovery [Part 3] Descriptor Calculation and Dataset Preparation**

**MOUNSIF EL ATOUCH**

In this Jupyter notebook, we will be building a machine learning model using the ChEMBL bioactivity data.

In **Part 3**, we will be calculating molecular descriptors that are essentially quantitative description of the compounds in the dataset. Finally, we will be preparing this into a dataset for subsequent model building in Part 4.

---

## **Download PaDEL-Descriptor**

In [1]:
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.zip
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.sh

--2023-06-04 23:08:12--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25768637 (25M) [application/zip]
Saving to: ‘padel.zip’

padel.zip           100%[===================>]  24.57M  --.-KB/s    in 0.08s   

2023-06-04 23:08:14 (309 MB/s) - ‘padel.zip’ saved [25768637/25768637]

--2023-06-04 23:08:14--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/padel/padel.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 231 [text/plain]
Saving to:

In [2]:
! unzip padel.zip

Archive:  padel.zip
   creating: PaDEL-Descriptor/
  inflating: __MACOSX/._PaDEL-Descriptor  
  inflating: PaDEL-Descriptor/MACCSFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._MACCSFingerprinter.xml  
  inflating: PaDEL-Descriptor/AtomPairs2DFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._AtomPairs2DFingerprinter.xml  
  inflating: PaDEL-Descriptor/EStateFingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._EStateFingerprinter.xml  
  inflating: PaDEL-Descriptor/Fingerprinter.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._Fingerprinter.xml  
  inflating: PaDEL-Descriptor/.DS_Store  
  inflating: __MACOSX/PaDEL-Descriptor/._.DS_Store  
   creating: PaDEL-Descriptor/license/
  inflating: __MACOSX/PaDEL-Descriptor/._license  
  inflating: PaDEL-Descriptor/KlekotaRothFingerprintCount.xml  
  inflating: __MACOSX/PaDEL-Descriptor/._KlekotaRothFingerprintCount.xml  
  inflating: PaDEL-Descriptor/config  
  inflating: __MACOSX/PaDEL-Descriptor/._config  
  inf

## **Installing librairies**

In [3]:
! pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 70.0 MB/s eta 0:00:00


## **Importing libraires**

In [4]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## **Load bioactivity data**

Download the curated ChEMBL bioactivity data that has been pre-processed from Parts 1 and 2 of this Bioinformatics Project series. Here we will be using the **bioactivity_data_curated.csv** file that essentially contain the pIC50 values that we will be using for building a classification model.

In [5]:
! wget https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_curated.csv

--2023-06-04 23:08:28--  https://raw.githubusercontent.com/mounsifelatouch/cdd/master/data/bioactivity_data_curated.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42420 (41K) [text/plain]
Saving to: ‘bioactivity_data_curated.csv’

bioactivity_data_cu 100%[===================>]  41.43K  --.-KB/s    in 0.003s  

2023-06-04 23:08:28 (11.6 MB/s) - ‘bioactivity_data_curated.csv’ saved [42420/42420]



In [6]:
df = pd.read_csv('bioactivity_data_curated.csv')

In [7]:
selection = ['canonical_smiles','chembl_id']
df_selection = df[selection]
df_selection.to_csv('molecule.smi', sep='\t', index=False, header=False)

In [8]:
! cat molecule.smi | head -5

O=c1oc2ccccc2c(O)c1Cc1c(O)c2ccccc2oc1=O	CHEMBL1466
O=c1oc2ccccc2c(O)c1C(c1ccc[nH]1)c1c(O)c2ccccc2oc1=O	CHEMBL260998
CCOc1cc(C(c2c(O)c3ccccc3oc2=O)c2c(O)c3ccccc3oc2=O)ccc1O	CHEMBL260997
O=c1oc2ccccc2c(O)c1C(c1c(O)c2ccccc2oc1=O)c1c[nH]c2ccccc12	CHEMBL258733
O=c1c(C(c2ccc([N+](=O)[O-])cc2)c2c(O)oc3ccccc3c2=O)c(O)oc2ccccc12	CHEMBL81935


## **Calculate fingerprint descriptors**


### **Calculate PaDEL descriptors**

In [9]:
! cat padel.sh

java -Xms1G -Xmx1G -Djava.awt.headless=true -jar ./PaDEL-Descriptor/PaDEL-Descriptor.jar -removesalt -standardizenitro -fingerprints -descriptortypes ./PaDEL-Descriptor/PubchemFingerprinter.xml -dir ./ -file descriptors_output.csv


In [ ]:
%%time
! bash padel.sh

## **Preparing the X and Y Data Matrices**

### **X data matrix**

In [57]:
df_X = pd.read_csv('descriptors_output.csv')

In [58]:
df_X = df_X.drop(columns=['Name'])

## **Y variable: Transformation into a binary variable**
The bioactivity data is in the pIC50 unit. Compounds having values of >= 6 will be considered to be **active = `1`** while those <= 5 will be considered to be **inactive = `0`**.

In [59]:
def get_class(y, thres=6):
  labels = {0: '0', 1: '1'}
  return labels[1] if y >= thres else labels[0]

In [60]:
df_Y = df['pIC50']

In [61]:
df_Y = df_Y.apply(get_class)

## **Combining X and Y variable**

In [62]:
data = pd.concat([df_X, df_Y], axis=1)

In [63]:
data = data.rename(columns={'pIC50': 'activity'})

In [64]:
data.to_csv('bioactivity_data_PubchemFingerprinter.csv', index=False)

In [65]:
! zip lab3.zip *

updating: bioactivity_data_curated.csv (deflated 76%)
updating: bioactivity_data_PubchemFingerprinter.csv (deflated 96%)
updating: descriptors_output.csv (deflated 96%)
updating: __MACOSX/ (stored 0%)
updating: molecule.smi (deflated 82%)
updating: PaDEL-Descriptor/ (stored 0%)
updating: padel.sh (deflated 36%)
updating: padel.zip (stored 0%)
updating: sample_data/ (stored 0%)
updating: bioactivity_data_PubchemFingerprinter-true.csv (deflated 96%)
